  MOVIE RECOMMENDATION SYSTEM USING  Auto Encoders

In [2]:
import numpy as np
import pandas as pd

import torch
import torch.nn as  nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
movies=pd.read_csv('movies.dat',sep='::',header=None,engine='python',encoding='latin-1')

In [4]:
users=pd.read_csv('users.dat',sep='::',header=None,engine='python',encoding='latin-1')

In [5]:
ratings=pd.read_csv('ratings.dat',sep='::',header=None,engine='python',encoding='latin-1')

In [6]:
training_set=pd.read_csv('u1.base',delimiter='\t')

In [7]:
print(training_set)

         1   1.1  5  874965758
0        1     2  3  876893171
1        1     3  4  878542960
2        1     4  3  876893119
3        1     5  3  889751712
4        1     7  4  875071561
...    ...   ... ..        ...
79994  943  1067  2  875501756
79995  943  1074  4  888640250
79996  943  1188  3  888640250
79997  943  1228  3  888640275
79998  943  1330  3  888692465

[79999 rows x 4 columns]


In [8]:
training_set=np.array(training_set,dtype='int')

In [9]:
test_set=pd.read_csv('u1.test',delimiter='\t')
test_set=np.array(test_set,dtype='int')

In [10]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))


In [11]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [12]:
training_set=torch.FloatTensor(training_set)
test_set=torch.FloatTensor(test_set)

CREATING STRUCTURE OF NEURAL NETWORKS

In [13]:
class SAE(nn.Module):
  def __init__(self,):
    super(SAE,self).__init__()
    self.fc1=nn.Linear(nb_movies,20)
    self.fc2=nn.Linear(20,10)
    self.fc3=nn.Linear(10,20)
    self.fc4=nn.Linear(20,nb_movies)
    self.activation= nn.Sigmoid()
  def forward(self,x):
    x=self.activation(self.fc1(x))
    x=self.activation(self.fc2(x))
    x=self.activation(self.fc3(x))
    x=self.fc4(x)
    return x


In [14]:
sae=SAE()

In [15]:
criterion=nn.MSELoss()
optimizer=optim.RMSprop(sae.parameters(),lr=0.01,weight_decay=0.5)


TRAINING THE SAE

In [17]:
nb_epoch=200
for epoch in range(1,nb_epoch+1):
  train_loss=0
  s=0.
  for id_user in range(nb_users):
    input=Variable(training_set[id_user]).unsqueeze(0)
    target=input.clone()
    if torch.sum(target.data>0)>0:
      output=sae(input)
      target.require_grad=False
      output[target==0]=0
      loss=criterion(output,target)
      mean_corrector=nb_movies/float(torch.sum(target.data>0)+1e-10)
      loss.backward()
      train_loss+=np.sqrt(loss.data*mean_corrector)
      s+=1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+str(train_loss/s))


/tmp/ipython-input-17-2148035191.py:15: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  train_loss+=np.sqrt(loss.data*mean_corrector)


epoch: 1loss: tensor(1.7720)
epoch: 2loss: tensor(1.0965)
epoch: 3loss: tensor(1.0533)
epoch: 4loss: tensor(1.0384)
epoch: 5loss: tensor(1.0308)
epoch: 6loss: tensor(1.0265)
epoch: 7loss: tensor(1.0235)
epoch: 8loss: tensor(1.0220)
epoch: 9loss: tensor(1.0207)
epoch: 10loss: tensor(1.0198)
epoch: 11loss: tensor(1.0187)
epoch: 12loss: tensor(1.0184)
epoch: 13loss: tensor(1.0179)
epoch: 14loss: tensor(1.0174)
epoch: 15loss: tensor(1.0171)
epoch: 16loss: tensor(1.0166)
epoch: 17loss: tensor(1.0169)
epoch: 18loss: tensor(1.0165)
epoch: 19loss: tensor(1.0164)
epoch: 20loss: tensor(1.0160)
epoch: 21loss: tensor(1.0161)
epoch: 22loss: tensor(1.0158)
epoch: 23loss: tensor(1.0158)
epoch: 24loss: tensor(1.0158)
epoch: 25loss: tensor(1.0158)
epoch: 26loss: tensor(1.0157)
epoch: 27loss: tensor(1.0154)
epoch: 28loss: tensor(1.0152)
epoch: 29loss: tensor(1.0126)
epoch: 30loss: tensor(1.0117)
epoch: 31loss: tensor(1.0093)
epoch: 32loss: tensor(1.0094)
epoch: 33loss: tensor(1.0052)
epoch: 34loss: tens

TESTING A SAE

In [18]:
test_loss=0
s=0.
for id_user in range(nb_users):
    input=Variable(training_set[id_user]).unsqueeze(0)
    target=input.clone()
    if torch.sum(target.data>0)>0:
      output=sae(input)
      target.require_grad=False
      output[target==0]=0
      loss=criterion(output,target)
      mean_corrector=nb_movies/float(torch.sum(target.data>0)+1e-10)

      train_loss+=np.sqrt(loss.data*mean_corrector)
      s+=1.

print('loss: '+str(test_loss/s))


/tmp/ipython-input-18-1469485880.py:13: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  train_loss+=np.sqrt(loss.data*mean_corrector)


loss: 0.0
